In [ ]:
import numpy as np 
import pandas as pd 
import os, gc, re, warnings
warnings.filterwarnings("ignore")

In [ ]:
dftr = pd.read_csv("/kaggle/input/amazon-ml/dataset/train.csv")
dftr["src"]="train"
dfte = pd.read_csv("/kaggle/input/amazon-ml/dataset/test.csv")
dfte["src"]="test"
print('Train shape:',dftr.shape,'Test shape:',dfte.shape,'Test columns:',dfte.columns)
df = pd.concat([dftr,dfte],ignore_index=True)

dftr.head()

In [ ]:
print(dftr.shape)
dftr = dftr.dropna(axis=0, subset=['TITLE'])
print(dftr.shape)

In [ ]:
dftr.fillna("", inplace=True)
dfte.fillna("", inplace=True)

In [ ]:
dftr.isna().sum()

In [ ]:
dfte.isna().sum()

In [ ]:
dfte.shape

In [ ]:
target_cols = ['PRODUCT_LENGTH']

In [ ]:
# import sys
# sys.path.append('../input/iterativestratification')
# from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
# FOLDS = 25
# skf = MultilabelStratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=42)
# for i,(train_index, val_index) in enumerate(skf.split(dftr,dftr[target_cols])):
#     dftr.loc[val_index,'FOLD'] = i
# print('Train samples per fold:')
# dftr.FOLD.value_counts()

# Generate Embeddings

In [ ]:
from transformers import AutoModel,AutoTokenizer, DataCollatorWithPadding
import torch
import torch.nn.functional as F
from tqdm import tqdm

In [ ]:
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output.last_hidden_state.detach().cpu()
    input_mask_expanded = (
        attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    )
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(
        input_mask_expanded.sum(1), min=1e-9
    )

In [ ]:
BATCH_SIZE = 4
tokenizer = None
MAX_LEN = 128

class EmbedDataset(torch.utils.data.Dataset):
    def __init__(self,df):
        self.df = df.reset_index(drop=True)
    def __len__(self):
        return len(self.df)
    def __getitem__(self,idx):
        text = self.df.loc[idx,"TITLE"]
        tokens = tokenizer(
                text,
                None,
                add_special_tokens=True,
                padding=False,
                truncation=True,
                max_length=32,return_tensors="pt")
        tokens = {k:v.squeeze(0) for k,v in tokens.items()}
        return tokens



In [ ]:
def get_embeddings(MODEL_NM='', MAX=640, BATCH_SIZE=4, verbose=True):
    global tokenizer, MAX_LEN
    DEVICE="cuda"
    model = AutoModel.from_pretrained( MODEL_NM )
    tokenizer = AutoTokenizer.from_pretrained( MODEL_NM )
    MAX_LEN = MAX
    
    ds_tr = EmbedDataset(dftr)
    embed_dataloader_tr = torch.utils.data.DataLoader(ds_tr,\
                        batch_size=BATCH_SIZE,\
                        shuffle=False, collate_fn=DataCollatorWithPadding(tokenizer))
    ds_te = EmbedDataset(dfte)
    embed_dataloader_te = torch.utils.data.DataLoader(ds_te,\
                        batch_size=BATCH_SIZE,\
                        shuffle=False, collate_fn=DataCollatorWithPadding(tokenizer))
    
    model = model.to(DEVICE)
    model.eval()
    all_train_text_feats = []
    for batch in tqdm(embed_dataloader_tr,total=len(embed_dataloader_tr)):
        input_ids = batch["input_ids"].to(DEVICE)
        attention_mask = batch["attention_mask"].to(DEVICE)
        with torch.no_grad():
            model_output = model(input_ids=input_ids,attention_mask=attention_mask)
        sentence_embeddings = mean_pooling(model_output, attention_mask.detach().cpu())
        # Normalize the embeddings
        sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)
        sentence_embeddings =  sentence_embeddings.squeeze(0).detach().cpu().numpy()
        all_train_text_feats.extend(sentence_embeddings)
    all_train_text_feats = np.array(all_train_text_feats)
    if verbose:
        print('Train embeddings shape',all_train_text_feats.shape)
        
    te_text_feats = []
    for batch in tqdm(embed_dataloader_te,total=len(embed_dataloader_te)):
        input_ids = batch["input_ids"].to(DEVICE)
        attention_mask = batch["attention_mask"].to(DEVICE)
        with torch.no_grad():
            model_output = model(input_ids=input_ids,attention_mask=attention_mask)
        sentence_embeddings = mean_pooling(model_output, attention_mask.detach().cpu())
        # Normalize the embeddings
        sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)
        sentence_embeddings =  sentence_embeddings.squeeze(0).detach().cpu().numpy()
        te_text_feats.extend(sentence_embeddings)
    te_text_feats = np.array(te_text_feats)
    if verbose:
        print('Test embeddings shape',te_text_feats.shape)
        
    return all_train_text_feats, te_text_feats

In [ ]:
MODEL_NM = '../input/huggingface-deberta-variants/deberta-base/deberta-base'
all_train_text_feats, te_text_feats = get_embeddings(MODEL_NM, MAX=32, BATCH_SIZE=32)

In [ ]:
np.save('train_deberta_base_feats.npy', all_train_text_feats)
np.save('test_deberta_base_feats.npy', te_text_feats)

In [ ]:
del all_train_text_feats, te_text_feats
gc.collect()

In [ ]:
MODEL_NM = '../input/deberta-v3-large/deberta-v3-large'
all_train_text_feats2, te_text_feats2 = get_embeddings(MODEL_NM, MAX=32, BATCH_SIZE=32)

In [ ]:
np.save('train_deberta_largev3_feats.npy', all_train_text_feats2)
np.save('test_deberta_largev3_feats.npy', te_text_feats2)

In [ ]:
del all_train_text_feats2, te_text_feats2
gc.collect()